In [ ]:
import numpy as np
import clip
import pandas as pd

In [ ]:
device =  "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [3]:
breeds = [
            "a dachshund",
            "a borzoi",
            "a basset hound",
            "a spitz",
            "a yorkshire terrier",
            "a chihuahua",
            "a german shepherd",
            "a labrador",
            "a husky",
            "a jack russell terrier",
            "a caucasian shepherd dog",
            "a corgi",
            "a bulldog",
            "a pug",
            "a poodle",
            "a golden retriever",
            "a rottweiler",
            "a beagle",
            "a cocker spaniel",
            "a deutscher boxer",
            "a pekingese",
            "a sharpey",
            "a bull terrier",
]

In [ ]:
encoded_text = model.encode_text(text)

In [ ]:
matrix_distance = []
for i in range(len(encoded_text)):
  s = []
  for j in range(len(encoded_text)):
    s.append(torch.mean((encoded_text[i] - encoded_text[j]) ** 2).item())
  matrix_distance.append(s)
matrix_distance = np.array(matrix_distance) / np.max(matrix_distance)
print(matrix_distance.shape)

In [ ]:
breeds_idx = {breeds[i]: i for i in range(len(breeds))}
breeds_idx

In [ ]:
def metric_color(color, real):
  if (color == real): return 0
  return 1

def metric_tail(tail, real):
  if (tail == real): return 0
  return 1

def metric_breed(breeds, real, breeds_distance, breeds_idx):
  breed = breeds.split(',')[0]
  return breeds_distance[breeds_idx[breed]][breeds_idx[real]]

def get_coord(cam):
  return np.random.randn(2)

def metric_dist(cam, real):
  diam = 50
  r1, r2 = get_coord(cam), get_coord(real)
  return np.sqrt((r1[0] - r2[0]) ** 2 + (r1[1] - r2[1]) ** 2) / diam

def find_most_sim_dog(csv_all: str, csv_sample: str, breeds_distance,
                      breeds_idx):
  weights = [0.5, 0.5, 1, 0.1] # tail, color, breed, distance
  data = pd.read_csv(csv_all, index_col = 0)[
    ["color", "tail", "top3_breed", "cam_id"]
  ].to_numpy()
  real = pd.read_csv(csv_all, index_col = 0)[
    ["color", "tail", "top3_breed", "cam_id"]
  ].to_numpy()  
  dist = []
  for inf in data:
    metrics = [metric_color(inf[0], real[0]), metric_tail(inf[1], real[1]),
               metric_breed(inf[2], real[2], breeds_distance, breeds_idx),
               metric_dist(inf[3], real[3])] 
    dist.append(sum([m * w] for (m, w) in zip(mterics, weight)))
  return(np.argmax(dist))